***
***

<img src="https://user-images.githubusercontent.com/8030363/106307643-d3fb4c00-621c-11eb-9c22-c2b7329db4a8.png" style="vertical-align:middle">

## Working with RDF Graphs

***

**Author:** [TJCallahan](http://tiffanycallahan.com/)  
**GitHub Repository:** [PheKnowLator](https://github.com/callahantiff/PheKnowLator/wiki)  
**Wiki Page:** [OWL-NETS-2.0](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0)  
**Release:** **[v2.0.0](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)**  
  
<br> 

### Purpose  
The goal of this notebook is to provide some examples of how to manipulate RDF ([Resource Description Framework](https://www.w3.org/RDF/)) graphs using the Python libraries [RDFLib](https://rdflib.readthedocs.io/en/stable/_modules/rdflib/compare.html) and [NetworkX](https://networkx.org/). This tutorial will cover the following concepts:  
- [Environment Set-Up](#setup)   
- [Loading Data](#data-load)     
- [Exploring Graphs](#exploring-graphs)  
- [Saving, Serializing, and Writing Graphs](#saving-output)    
- [Try it Yourself](#try-it-yourself)  


#### Resources   
- RDFLib  
  - [Getting started with RDFLib](https://rdflib.readthedocs.io/en/stable/gettingstarted.html)   
- NetworkX
  - [Introduction to NetworkX](https://networkx.org/documentation/stable//reference/introduction.html)  
  - [Cambridge NetworkX Tutorial](https://www.cl.cam.ac.uk/teaching/1314/L109/tutorial.pdf)  
- SPARQL Query
  - [Constructing SPARQL Queries](https://www.w3.org/2009/Talks/0615-qbe/)   
  - [SPARQL - Medium Article](https://medium.com/wallscope/constructing-sparql-queries-ca63b8b9ac02)


<br>

***  
### Set-Up Environment <a class="anchor" id="setup"></a>
***  

#### Dependencies: [`pkt_kg`](https://pypi.org/project/pkt-kg/), [`networkx`](https://pypi.org/project/networkx/), [`rdflib`](https://pypi.org/project/rdflib/)

To prepare for the tutorial we need to make sure that the all needed libraries are downloaded and imported. If you don't already have `pkt_kg`, `networkx` and `rdflib` installed, you can uncomment the chunk shown below and replace `<<MODULE NAME>>` with each of the libraries that you need to download. In addition to downloading needed libraries, you will also need to download graph data. Each data source is briefly described in the next section.  

In [ ]:
# # uncomment and update to install any required modules
# import sys
# !{sys.executable} -m pip install <<MODULE NAME>>

In [7]:
# import needed libraries
import networkx as nx
import os

from rdflib import Graph, Namespace, URIRef, BNode, Literal
from tqdm import tqdm

from pkt_kg.utils import *

#### Tutorial Data  
This tutorial utilizes two separate graphs. The first graph is the [Vaccine Ontology (VO)](http://www.obofoundry.org/ontology/vo.html) and the second graph is one of the [`PheKnowlator`](https://github.com/callahantiff/PheKnowLator) builds. Each of these data sources are briefly described below:  

**[`Vaccine Ontology`](http://purl.obolibrary.org/obo/vo.owl)**: For this resource, we will download and utilize the [OWL](https://www.w3.org/TR/owl-features/) file provided by the Open Biomedical Ontologies Foundry. The link used to download this ontology will ensure that no matter when    

**[`PheKnowLator Build`](https://storage.googleapis.com/pheknowlator/archived_builds/release_v2.0.0/build_11FEB2021/knowledge_graphs/instance_builds/relations_only/owlnets/PheKnowLator_v2.0.0_full_instance_relationsOnly_noOWL_OWLNETS.nt)**: For this resources, we will utilize data from the `PheKnowLator` instance build, with directed relations, and [`OWL-NETS`](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0) property graph conversion (`PheKnowLator_v2.0.0_full_instance_relationsOnly_noOWL_OWLNETS.nt`). To demonstrate different ways that data can be processed, we will be downloading the data in the [`ntriples`](https://www.w3.org/TR/n-triples/) format.   


In [10]:
# Set global variables
write_location = 'temp_dir/'
if not os.path.exists(write_location):
    os.mkdir(write_location)

In [12]:
# download data to the temp_dir directory
data_urls = [
    'http://purl.obolibrary.org/obo/vo.owl',
    'https://storage.googleapis.com/pheknowlator/archived_builds/release_v2.0.0/build_25JAN2021/knowledge_graphs/instance_builds/relations_only/owlnets/PheKnowLator_v2.0.0_full_instance_relationsOnly_noOWL_OWLNETS.nt'
]

for url in data_urls:
    file_name = url.split('/')[-1]
    if not os.path.exists(write_location + file_name):
        data_downloader(url, write_location)

<br>

***
### Loading Data <a class="anchor" id="data-load"></a>
***

**Relevant Documentation:** [`https://rdflib.readthedocs.io/en/stable/intro_to_parsing.html`](https://rdflib.readthedocs.io/en/stable/intro_to_parsing.html)  

This section demonstrates how to load data using `RDFLib`. Please note that there are many different ways that one can load data, which largely depends on the the format of the data you are trying to import. See the link above for details regarding the different types of formats that the library can import.

In this tutorial we will be importing data to an `RDFLib` `Graph()` object using two different imports formats: (1) `vo.owl` using `xml` format; and (2) `PheKnowLator_v2.0.0_full_instance_relationsOnly_noOWL_OWLNETS.nt` using `nt` format.

##### Loading `vo.owl`

In [14]:
vo_graph = Graph().parse(write_location + 'vo.owl', format='xml')

<br>

***
### Exploring Graphs <a class="anchor" id="exploring-graphs"></a>
***

Before diving into this tutorial, it's useful to review some of the tutorials on accessing and manipulating `RDFLib` `Graph()` objects. Specifically, it's useful to review the following:  
- **[RDFLib Terms](https://rdflib.readthedocs.io/en/stable/rdf_terms.html?highlight=terms)**: `URIRef()`, `Literal()`, and `BNode()`  
- **[Triple Pattern Matching](https://rdflib.readthedocs.io/en/stable/intro_to_graphs.html#basic-triple-matching)**: `graph.triples()`, `graph.subjects()`, `graph.objects()`, and `graph.predicates()`  
- **[Namespace Utilities](https://rdflib.readthedocs.io/en/stable/apidocs/rdflib.html?highlight=namespace#namespace-utilities)**: Provides description of the built-in namespaces (e.g. `OWL`, `RDF`, `RDFS`, etc...)  
- **[SPARQL Queries](https://rdflib.readthedocs.io/en/stable/intro_to_sparql.html):** `graph.query()`

In [29]:
# import built-in namespaces
from rdflib.namespace import OWL, RDF, RDFS


# create namespace
obo = Namespace('http://purl.obolibrary.org/obo/')

#### Graph Descriptives 

The code chunks below provide different examples of how to obtain counts of different parts of the graph. Specifically, the code below demonstrates how to obtain counts of `owl:Class` and `owl:ObjectProperty`.

*Approach 1:* Obtain counts using a SPARQL query. `RDFLib` documentation for writing SPARQL queries can be found [here](https://rdflib.readthedocs.io/en/stable/intro_to_sparql.html). 

In [16]:
# Approach 1: SPARQL query -- note the use of OWL and RDF built-in namespaces
class_query = vo_graph.query(
    """SELECT DISTINCT ?class
    WHERE {
      ?class rdf:type owl:Class .}
    """, initNs={'rdf': RDF,
                 'owl': OWL}) 

print('There are {} owl:Class objects'.format(len(class_query)))

There are 7089 owl:Class objects


In [27]:
object_query = vo_graph.query(
    """SELECT DISTINCT ?class
    WHERE {
      ?class rdf:type owl:ObjectProperty .}
    """, initNs={'rdf': RDF,
                 'owl': OWL}) 

print('There are {} owl:ObjectProperties objects'.format(len(object_query)))

There are 232 owl:ObjectProperties objects


*Approach 2:* Obtain counts using `RDFLib` built-in methods. Specifically, the examples below demonstrate the `subjects()` and `triples()` methods. `RDFLib` documentation for these built-in functions can be found [here](https://rdflib.readthedocs.io/en/stable/intro_to_graphs.html#graph-methods-for-accessing-triples). 

In [21]:
# Approach 2: Iterate over RDFLib graph object
classes = list(vo_graph.subjects(RDF.type, OWL.Class))
objects = list(vo_graph.subjects(RDF.type, OWL.ObjectProperty))
triples = list(vo_graph.triples((None, None, None)))

print('There are {} triples, {} owl:Class objects, and {} owl:ObjectProperties objects'.format(len(triples),
                                                                                               len(classes),
                                                                                               len(objects)))

There are 86454 triples, 7089 owl:Class objects, and 232 owl:ObjectProperties objects


#### Helpful Functions
The next code sections provide examples of how to access different types of information from an `RDF` graph.

In [31]:
# get labels for the URI VO_0002186 -- see two different ways to reference a URI
# using obo namespace
vo_0002186_label = vo_graph.label(obo.VO_0002186)
print(str(vo_0002186_label))

Newcastle-Bronchitis VG/GA Strain, Mass & Conn Types, Live Virus Vaccine (USDA: 17V1.11)


In [32]:
# using RDFLIb URIRef term
vo_0002186_label = vo_graph.label(URIRef('http://purl.obolibrary.org/obo/VO_0002186'))
print(str(vo_0002186_label))

Newcastle-Bronchitis VG/GA Strain, Mass & Conn Types, Live Virus Vaccine (USDA: 17V1.11)


In [34]:
# get all triples that VO_0002186 participates in
VO_0002186_triples = list(vo_graph.triples((obo.VO_0002186, None, None))) +\
                     list(vo_graph.triples((obo.VO_0002186, None, None)))

for s, p, o in tqdm(VO_0002186_triples):
    print(s, p, o)

100%|██████████| 20/20 [00:00<00:00, 7770.11it/s]

http://purl.obolibrary.org/obo/VO_0002186 http://www.w3.org/2000/01/rdf-schema#subClassOf N5f04b1e00ae44669aab184ebb3b4bacb
http://purl.obolibrary.org/obo/VO_0002186 http://purl.obolibrary.org/obo/IAO_0000116 generated using OntoRat
http://purl.obolibrary.org/obo/VO_0002186 http://www.w3.org/2000/01/rdf-schema#label Newcastle-Bronchitis VG/GA Strain, Mass & Conn Types, Live Virus Vaccine (USDA: 17V1.11)
http://purl.obolibrary.org/obo/VO_0002186 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
http://purl.obolibrary.org/obo/VO_0002186 http://www.w3.org/2000/01/rdf-schema#subClassOf http://purl.obolibrary.org/obo/VO_0002457
http://purl.obolibrary.org/obo/VO_0002186 http://www.w3.org/2000/01/rdf-schema#subClassOf Nf08ba96cc3cb42bda3d12dd44e4c7948
http://purl.obolibrary.org/obo/VO_0002186 http://www.w3.org/2000/01/rdf-schema#seeAlso violinID:2066;2067
http://purl.obolibrary.org/obo/VO_0002186 http://www.w3.org/2000/01/rdf-schema#subClassOf N0f716452b9ac40

In [38]:
# get all owl:ObjectProperty objects
vo_obj_props = list(vo_graph.subjects(RDF.type, OWL.ObjectProperty))

for p in tqdm(vo_obj_props):
    print(p)

100%|██████████| 232/232 [00:00<00:00, 32538.99it/s]

http://purl.obolibrary.org/obo/RO_0002087
http://purl.obolibrary.org/obo/OBI_0000295
http://purl.obolibrary.org/obo/RO_0002404
http://purl.obolibrary.org/obo/RO_0000085
http://purl.obolibrary.org/obo/RO_0002211
http://purl.obolibrary.org/obo/RO_0004006
http://purl.obolibrary.org/obo/RO_0004012
http://purl.obolibrary.org/obo/RO_0004014
http://purl.obolibrary.org/obo/VO_0000553
http://purl.obolibrary.org/obo/RO_0002508
http://purl.obolibrary.org/obo/RO_0002009
http://purl.obolibrary.org/obo/RO_0002431
http://purl.obolibrary.org/obo/VO_0000537
http://purl.obolibrary.org/obo/RO_0002511
http://purl.obolibrary.org/obo/RO_0002215
http://purl.obolibrary.org/obo/RO_0002597
http://purl.obolibrary.org/obo/RO_0004035
http://purl.obolibrary.org/obo/RO_0000086
http://purl.obolibrary.org/obo/VO_0000799
http://purl.obolibrary.org/obo/RO_00000087
http://purl.obolibrary.org/obo/VO_0003355
http://purl.obolibrary.org/obo/RO_0002206
http://purl.obolibrary.org/obo/RO_0002513
http://purl.obolibrary.org/obo/R

#### Obtaining Detailed Network Statistics  
While `RDFLib` is a great library for building `RDF` graphs, it lacks functionality to obtain some of the more traditional network statistics. In order to be able to use the `NetworkX` functions, we first need to convert the `RDFLib` `Graph()` object into a `NetworkX` [MultiDiGraph](https://networkx.github.io/documentation/stable/reference/classes/multidigraph.html). You will notice, if you read the `RDFLib` documentation that there are methods that will do this for you. In my experience, they are incredibly slow, it's much faster to use the code shown below.

From [this](https://networkx.org/documentation/stable/reference/classes/multidigraph.html) `NetworkX` documentation:  
> A directed graph class that can store multiedges.  
>  
> Multiedges are multiple edges between two nodes. Each edge can hold optional data or attributes.  
>  
> A MultiDiGraph holds directed edges. Self loops are allowed.  
>  
> Nodes can be arbitrary (hashable) Python objects with optional key/value attributes. By convention None is not used as a node. 
>  
> Edges are represented as links between nodes with optional key/value attributes.

In [39]:
# convert RDFLib graph to Networkx MultiDiGraph
nx_graph = nx.MultiDiGraph()

for s, p, o in tqdm(vo_graph):
    nx_graph.add_edge(s, o, **{'key': p})

100%|██████████| 86454/86454 [00:02<00:00, 38869.71it/s]


In [43]:
# get the number of nodes, edges, and self-loops
nodes = nx.number_of_nodes(nx_graph)
edges = nx.number_of_edges(nx_graph)
self_loops = nx.number_of_selfloops(nx_graph)

print('There are {} nodes, {} edges, and {} self-loop(s)'.format(nodes, edges, self_loops))

There are 40539 nodes, 86454 edges, and 1 self-loops


In [45]:
# get degree information
avg_degree = float(edges) / nodes

print('The Average Degree is {}'.format(avg_degree))

The Average Degree is 2.132613039295493


In [53]:
# get 5 nodes with the highest degress
n_deg = sorted([(str(x[0]), x[1]) for x in  nx_graph.degree], key=lambda x: x[1], reverse=1)[:6]

for x in n_deg:
    print('{} (degree={})'.format(x[0], x[1]))

http://www.w3.org/2002/07/owl#Restriction (degree=8755)
http://www.w3.org/2002/07/owl#Class (degree=7089)
http://purl.obolibrary.org/obo/RO_0000087 (degree=1917)
http://purl.obolibrary.org/obo/VO_0003355 (degree=1456)
http://purl.obolibrary.org/obo/OBI_0000304 (degree=1453)
http://purl.obolibrary.org/obo/BFO_0000051 (degree=1247)


In [55]:
# get network density
density = nx.density(nx_graph)

print('The density of the graph is: {}'.format(density))

The density of the graph is: 5.260775172173006e-05


In [68]:
# get connected components -- have to convert MultiDiGraph to undirected graph
nx_graph_und = nx_graph.to_undirected()

# get connected components
components = sorted(list(nx.connected_components(nx_graph_und)), key=len, reverse=True)
cc_content = {x: str(len(components[x])) + ' nodes: ' + ' | '.join(components[x]) if len(components[x]) < 500
              else len(components[x]) for x in range(len(components))}

for k, v in cc_content.items():
    if isinstance(v, int):
        print('COMPONENT: {} Consists of {} nodes'.format(str(k), str(v)))
    else:
        print('\nCOMPONENT: {} Consists of the following nodes:'.format(str(k)))
        for node in v.split(': ')[-1].split(' | '):
            print(node)

COMPONENT: 0 Consists of 40516 nodes

COMPONENT: 1 Consists of the following nodes:
http://purl.obolibrary.org/obo/CARO_0001001
http://purl.obolibrary.org/obo/CARO_0001000
http://purl.obolibrary.org/obo/caro.owl
http://purl.obolibrary.org/obo/CARO_0010000
http://purl.obolibrary.org/obo/CARO_0000055
http://purl.obolibrary.org/obo/CARO_0000003
http://purl.obolibrary.org/obo/CARO_0000014
http://purl.obolibrary.org/obo/CARO_0000043
http://purl.obolibrary.org/obo/CARO_0000006

COMPONENT: 2 Consists of the following nodes:
http://www.w3.org/2000/01/rdf-schema#Literal
http://www.w3.org/2001/XMLSchema#string
http://www.w3.org/2000/01/rdf-schema#Datatype
http://www.w3.org/2001/XMLSchema#float
http://www.w3.org/2001/XMLSchema#nonNegativeInteger
http://www.w3.org/2001/XMLSchema#date
http://www.w3.org/2001/XMLSchema#dateTime
http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral
http://www.w3.org/2001/XMLSchema#integer

COMPONENT: 3 Consists of the following nodes:
http://purl.obolibrary.org/obo/

##### Shortest Path

This chunk shows how to find the shortest path between a starting node and all nodes reachable from that node. From [this](https://networkx.org/documentation/stable//reference/algorithms/generated/networkx.algorithms.shortest_paths.unweighted.single_source_shortest_path.html#networkx.algorithms.shortest_paths.unweighted.single_source_shortest_path) `NetworkX` page:  

> Compute shortest path between source and all other nodes reachable from source.
> 
> **Parameters:**
>   - G (NetworkX graph)
>   - source (node label) – Starting node for path
>   - cutoff (integer, optional) – Depth to stop the search. Only paths of length <= cutoff are returned.
> 
> **Returns:**   
>   lengths – Dictionary, keyed by target, of shortest paths.  
>
> **Return type:**  
> dictionary

In [133]:
# get shortest path from VO_0002186
vo_0002186_path = nx.single_source_shortest_path(nx_graph, obo.VO_0002186)

for k, v in vo_0002186_path.items():
    if k != obo.VO_0002186:
        print('\n{} - {} Path:'.format(str(obo.VO_0002186).split('/')[-1], str(k).split('/')[-1]))
        for i in v:
            print(i)


VO_0002186 - violinID:2066;2067 Path:
http://purl.obolibrary.org/obo/VO_0002186
violinID:2066;2067

VO_0002186 - USDA: 17V1.11 Path:
http://purl.obolibrary.org/obo/VO_0002186
USDA: 17V1.11

VO_0002186 - owl#Class Path:
http://purl.obolibrary.org/obo/VO_0002186
http://www.w3.org/2002/07/owl#Class

VO_0002186 - GA Strain, Mass & Conn Types, Live Virus Vaccine (USDA: 17V1.11) Path:
http://purl.obolibrary.org/obo/VO_0002186
Newcastle-Bronchitis VG/GA Strain, Mass & Conn Types, Live Virus Vaccine (USDA: 17V1.11)

VO_0002186 - generated using OntoRat Path:
http://purl.obolibrary.org/obo/VO_0002186
generated using OntoRat

VO_0002186 - N0f716452b9ac40baa94f6b47262c9e70 Path:
http://purl.obolibrary.org/obo/VO_0002186
N0f716452b9ac40baa94f6b47262c9e70

VO_0002186 - N5f04b1e00ae44669aab184ebb3b4bacb Path:
http://purl.obolibrary.org/obo/VO_0002186
N5f04b1e00ae44669aab184ebb3b4bacb

VO_0002186 - VO_0002457 Path:
http://purl.obolibrary.org/obo/VO_0002186
http://purl.obolibrary.org/obo/VO_0002457



N5f04b1e00ae44669aab184ebb3b4bacb
http://purl.obolibrary.org/obo/RO_0000087
http://purl.obolibrary.org/obo/RO_0000053
http://purl.obolibrary.org/obo/BFO_0000020
A continuant  that inheres in or is borne by other entities. Every instance of A requires some specific instance of B which must always be the same.

VO_0002186 - SpecificallyDependentContinuant Path:
http://purl.obolibrary.org/obo/VO_0002186
N5f04b1e00ae44669aab184ebb3b4bacb
http://purl.obolibrary.org/obo/RO_0000087
http://purl.obolibrary.org/obo/RO_0000053
http://purl.obolibrary.org/obo/BFO_0000020
SpecificallyDependentContinuant

VO_0002186 - specifically dependent continuant Path:
http://purl.obolibrary.org/obo/VO_0002186
N5f04b1e00ae44669aab184ebb3b4bacb
http://purl.obolibrary.org/obo/RO_0000087
http://purl.obolibrary.org/obo/RO_0000053
http://purl.obolibrary.org/obo/BFO_0000020
specifically dependent continuant

VO_0002186 - the mutual dependence of proton donors and acceptors in chemical reactions [79 Path:
http://purl.o

http://purl.obolibrary.org/obo/VO_0002186
N0f716452b9ac40baa94f6b47262c9e70
http://purl.obolibrary.org/obo/VO_0000173
http://purl.obolibrary.org/obo/PATO_0002146
http://purl.obolibrary.org/obo/PATO_0001995
http://purl.obolibrary.org/obo/PATO_0001237
http://purl.obolibrary.org/obo/PATO_0001241
http://purl.obolibrary.org/obo/BFO_0000019
(forall (x) (if (Quality x) (SpecificallyDependentContinuant x))) // axiom label in BFO2 CLIF: [055-001] 

VO_0002186 -  axiom label in BFO2 CLIF: [105-001]  Path:
http://purl.obolibrary.org/obo/VO_0002186
N0f716452b9ac40baa94f6b47262c9e70
http://purl.obolibrary.org/obo/VO_0000173
http://purl.obolibrary.org/obo/PATO_0002146
http://purl.obolibrary.org/obo/PATO_0001995
http://purl.obolibrary.org/obo/PATO_0001237
http://purl.obolibrary.org/obo/PATO_0001241
http://purl.obolibrary.org/obo/BFO_0000019
(forall (x) (if (exists (t) (and (existsAt x t) (Quality x))) (forall (t_1) (if (existsAt x t_1) (Quality x))))) // axiom label in BFO2 CLIF: [105-001] 

VO_00021

http://purl.obolibrary.org/obo/VO_0002457
http://purl.obolibrary.org/obo/VO_0001222
http://purl.obolibrary.org/obo/VO_0000609
http://purl.obolibrary.org/obo/VO_0000001
N575e7339bb7b4e7bbe5ab5854c7d34c2
Nd5e1adeba9004c64aea5f1a7b1bfccef
N87257a7cc9f74c48be59fe7930505ba5
N29bda16e15f348c38988b8da95322dea
http://purl.obolibrary.org/obo/VO_0000590
vaccine preparation is a manufacturing process to produce a vaccine.

VO_0002186 - YH, BP Path:
http://purl.obolibrary.org/obo/VO_0002186
http://purl.obolibrary.org/obo/VO_0002457
http://purl.obolibrary.org/obo/VO_0001222
http://purl.obolibrary.org/obo/VO_0000609
http://purl.obolibrary.org/obo/VO_0000001
N575e7339bb7b4e7bbe5ab5854c7d34c2
Nd5e1adeba9004c64aea5f1a7b1bfccef
N87257a7cc9f74c48be59fe7930505ba5
N29bda16e15f348c38988b8da95322dea
http://purl.obolibrary.org/obo/VO_0000590
YH, BP

VO_0002186 - N6504ff4a563c4856b37f932913f2f234 Path:
http://purl.obolibrary.org/obo/VO_0002186
http://purl.obolibrary.org/obo/VO_0002457
http://purl.obolibrary.or

http://purl.obolibrary.org/obo/VO_0001222
http://purl.obolibrary.org/obo/VO_0000609
http://purl.obolibrary.org/obo/VO_0000001
N575e7339bb7b4e7bbe5ab5854c7d34c2
Nd5e1adeba9004c64aea5f1a7b1bfccef
N87257a7cc9f74c48be59fe7930505ba5
N29bda16e15f348c38988b8da95322dea
http://purl.obolibrary.org/obo/OBI_0000312
http://purl.obolibrary.org/obo/OBI_0000011
http://purl.obolibrary.org/obo/BFO_0000015
BFO 2 Reference: The realm of occurrents is less pervasively marked by the presence of natural units than is the case in the realm of independent continuants. Thus there is here no counterpart of ‘object’. In BFO 1.0 ‘process’ served as such a counterpart. In BFO 2.0 ‘process’ is, rather, the occurrent counterpart of ‘material entity’. Those natural – as contrasted with engineered, which here means: deliberately executed – units which do exist in the realm of occurrents are typically either parasitic on the existence of natural units on the continuant side, or they are fiat in nature. Thus we can count

http://purl.obolibrary.org/obo/VO_0000001
N575e7339bb7b4e7bbe5ab5854c7d34c2
Nd5e1adeba9004c64aea5f1a7b1bfccef
N87257a7cc9f74c48be59fe7930505ba5
N29bda16e15f348c38988b8da95322dea
http://purl.obolibrary.org/obo/VO_0000590
N9ec14bfeaa7c4c4fb1569994deeed21f
http://purl.obolibrary.org/obo/BFO_0000055
N01c3b0477a474e78991718cf66ee232b
http://purl.obolibrary.org/obo/BFO_0000054

VO_0002186 - opisthokonts Path:
http://purl.obolibrary.org/obo/VO_0002186
Nf08ba96cc3cb42bda3d12dd44e4c7948
http://purl.obolibrary.org/obo/OBI_0000304
N62e87a11dae7422aa64564496eca254c
Nb0ebe8f25a1e4f6ab4df3e8f34c5c8dd
http://purl.obolibrary.org/obo/NCBITaxon_9606
http://purl.obolibrary.org/obo/NCBITaxon_314146
http://purl.obolibrary.org/obo/NCBITaxon_40674
http://purl.obolibrary.org/obo/NCBITaxon_32524
http://purl.obolibrary.org/obo/NCBITaxon_117571
http://purl.obolibrary.org/obo/NCBITaxon_7742
http://purl.obolibrary.org/obo/NCBITaxon_33213
http://purl.obolibrary.org/obo/NCBITaxon_33154
opisthokonts

VO_0002186 - Opi

http://purl.obolibrary.org/obo/VO_0000609
http://purl.obolibrary.org/obo/VO_0000001
N575e7339bb7b4e7bbe5ab5854c7d34c2
Nd5e1adeba9004c64aea5f1a7b1bfccef
N87257a7cc9f74c48be59fe7930505ba5
Nd570873b16cb4bbdbfbccb3d1cc643ec
N262b7c57b9a44a4580c72170860f43b5
N675106ff5e28447c8b04eb69d19c7bd4
Nfa2a44fc21cf43f6ad1fad5433fe4636
http://purl.obolibrary.org/obo/VO_0000278
Nf9b38a6652db4e80a812cd1ddd387fc6
N4ba8d98b123b45d2ae96e30fd3c1b900
Nc1d4182644344757a86d40be5280c72c
http://purl.obolibrary.org/obo/VO_0000562

VO_0002186 - action specification Path:
http://purl.obolibrary.org/obo/VO_0002186
http://purl.obolibrary.org/obo/VO_0002457
http://purl.obolibrary.org/obo/VO_0001222
http://purl.obolibrary.org/obo/VO_0000609
http://purl.obolibrary.org/obo/VO_0000001
N575e7339bb7b4e7bbe5ab5854c7d34c2
Nd5e1adeba9004c64aea5f1a7b1bfccef
N87257a7cc9f74c48be59fe7930505ba5
N29bda16e15f348c38988b8da95322dea
http://purl.obolibrary.org/obo/OBI_0000312
http://purl.obolibrary.org/obo/OBI_0000011
Nd392481f64fc42d0b8

A continuant cannot be part of an occurrent: use 'participates in'. An occurrent cannot be part of a continuant: use 'has participant'. A material entity cannot be part of an immaterial entity: use 'has location'. A specifically dependent continuant cannot be part of an independent continuant: use 'inheres in'. An independent continuant cannot be part of a specifically dependent continuant: use 'bearer of'.

VO_0002186 - my brain is part of my body (continuant parthood, two material entities) Path:
http://purl.obolibrary.org/obo/VO_0002186
http://purl.obolibrary.org/obo/VO_0002457
http://purl.obolibrary.org/obo/VO_0001222
http://purl.obolibrary.org/obo/VO_0000609
http://purl.obolibrary.org/obo/VO_0000001
N575e7339bb7b4e7bbe5ab5854c7d34c2
Nd5e1adeba9004c64aea5f1a7b1bfccef
N87257a7cc9f74c48be59fe7930505ba5
N29bda16e15f348c38988b8da95322dea
http://purl.obolibrary.org/obo/OBI_0000312
http://purl.obolibrary.org/obo/OBI_0000011
Nd392481f64fc42d0b8cb838284fdf0f1
Nb5b30dc836134ec1adda0d232daa0

http://purl.obolibrary.org/obo/OBI_0600007
N716a7abf71eb4c968a5c16e39a5c98f3
N5c6aa0352bd2413ca87a373045565c0d
Nf6bd61019b5b4ce4bdc15e95aa426d69
Na1a576711cda458e80ed1560ba938c34
Ne9a06fb795fa4b679b13fb1b3e2b45c8
N2a8a1386d71246998b08b176bdfb5c13

VO_0002186 - OBI_0000444 Path:
http://purl.obolibrary.org/obo/VO_0002186
http://purl.obolibrary.org/obo/VO_0002457
http://purl.obolibrary.org/obo/VO_0001222
http://purl.obolibrary.org/obo/VO_0000609
http://purl.obolibrary.org/obo/VO_0000001
N575e7339bb7b4e7bbe5ab5854c7d34c2
Nd5e1adeba9004c64aea5f1a7b1bfccef
N87257a7cc9f74c48be59fe7930505ba5
Nd570873b16cb4bbdbfbccb3d1cc643ec
N262b7c57b9a44a4580c72170860f43b5
N675106ff5e28447c8b04eb69d19c7bd4
Nfa2a44fc21cf43f6ad1fad5433fe4636
http://purl.obolibrary.org/obo/VO_0000278
Nf9b38a6652db4e80a812cd1ddd387fc6
N4ba8d98b123b45d2ae96e30fd3c1b900
http://purl.obolibrary.org/obo/VO_0000452
Nec0de1fa952e46298d6b5c7a06f2fb1c
http://purl.obolibrary.org/obo/VO_0000002
N847c8290577746efa64ec4567ac67f6e
Nf2da5ffb61

N98286a727f2c48fe9cb807b4db792e15
Ncf7e22af64ac45eb9d2d1627677c2d62
N485e466f072944c1b361e6475d4c1271
Ne9e2ca170c184a29bfe5e70bf457e3dc

VO_0002186 - N72562b68052f4877ae33b065fd89fd53 Path:
http://purl.obolibrary.org/obo/VO_0002186
http://purl.obolibrary.org/obo/VO_0002457
http://purl.obolibrary.org/obo/VO_0001222
http://purl.obolibrary.org/obo/VO_0000609
http://purl.obolibrary.org/obo/VO_0000001
N575e7339bb7b4e7bbe5ab5854c7d34c2
Nd5e1adeba9004c64aea5f1a7b1bfccef
N87257a7cc9f74c48be59fe7930505ba5
Nd570873b16cb4bbdbfbccb3d1cc643ec
N262b7c57b9a44a4580c72170860f43b5
N675106ff5e28447c8b04eb69d19c7bd4
Nfa2a44fc21cf43f6ad1fad5433fe4636
http://purl.obolibrary.org/obo/VO_0000278
Nf9b38a6652db4e80a812cd1ddd387fc6
N4ba8d98b123b45d2ae96e30fd3c1b900
http://purl.obolibrary.org/obo/VO_0000452
Nec0de1fa952e46298d6b5c7a06f2fb1c
http://purl.obolibrary.org/obo/VO_0000002
N847c8290577746efa64ec4567ac67f6e
Nf2da5ffb619842f1a6cd064c6e6c78b8
Nf9d8cd55400b45e28197363c7c2e1be6
N6c24164e47714b06bf28453a56486c1

<br>

***
### Saving, Serializing, and Writing Graphs <a class="anchor" id="saving-output"></a>
***

Below, I provide examples for how to save data using `NetworkX` and `RDFLib`.

#### NetworkX  
**Documentation:** [Reading and Writing Data](https://networkx.org/documentation/stable//reference/readwrite/index.html?highlight=write%20data)

Here, I provide an example of how to save the `MultiDiGraph` version of the `vo_graph` object so it can be used in the future without having to create it from the `RDFLib` graph object.

In [72]:
# save multidigraph version of graph
nx.write_gpickle(nx_graph, write_location + 'vo_NetworkxMultiDiGraph.gpickle')

# read in multidigraph version of graph
# nx_graph = nx.read_gpickle(write_location + 'vo_NetworkxMultiDiGraph.gpickle')

#### RDFLib  
**Documentation:** [Graph Serialization](https://rdflib.readthedocs.io/en/latest/apidocs/rdflib.html#rdflib.graph.Graph.serialize)

Here, I provide an example of how to save the `vo_graph` object as an `OWL` file and as an `n-triples` file.

In [70]:
# save vo_graph as `ntriple` format
vo_graph.serialize(write_location + 'vo_graph_data.nt', format='nt')

In [71]:
# save vo_graph as OWL (i.e. RDF/XML)
vo_graph.serialize(write_location + 'vo_graph_data.owl', format='xml')

<br>

***
### Try it Out for Yourself - Explore the PheKnowLator Graph Build <a class="anchor" id="try-it-yourself"></a>
***

You will notice that while we downloaded the `pkt_kg` graph data `PheKnowLator_v2.0.0_full_instance_relationsOnly_noOWL_OWLNETS.nt`, we never used it. The remainder of this tutorial is left for you. I encourage you to use the functionality introduced above to explore this graph. If you do something interesting and want to share it, please make a pull-request and add it to this tutorial!

##### Loading `PheKnowLator_v2.0.0_full_instance_relationsOnly_noOWL_OWLNETS.nt`  

Please note that running the chunk below can take up to 45 min as the file is 2.0 GB.

In [80]:
pkt_graph = Graph().parse(write_location + 'PheKnowLator_v2.0.0_full_instance_relationsOnly_noOWL_OWLNETS.nt', format='nt')

In [81]:
# get basic descriptive statistics
triples = len(pkt_graph)
nodes = len(set(list(pkt_graph.subjects()) + list(pkt_graph.objects())))
rels = set(list(pkt_graph.predicates()))

# print stats
x = ' {} triples, {} nodes, {} predicates'
print('Graph Stats:' + x.format(triples, nodes, len(rels)))

Graph Stats: 13438644 triples, 1169694 nodes, 325 predicates, 0 classes, 0 individuals, 0 object properties, 0 annotation properties


##### Create Subgraph  

Create a subgraph from the `pkt_kg` build that only contains the following types of edges: `gene-drug`, `drug-disease`, `gene-disease`. To do this, we filter the graph first by relation and then by node types. Examples for each relation type are listed below:  

`drug-gene`: 
- Find all triples that include the predicate: *interacts with* (`obo:RO_0002434`)  
  - Only keep subjects with the following URI: `obo:CHEBI_XXXXXXX`  
  - Only keep objects with the following URI: `http://www.ncbi.nlm.nih.gov/gene/`

`drug-disease`:  
- Find all triples that include the predicate: *substance that treats* (`obo:RO_0002606`)  
  - Only keep subjects with the following URI: `obo:CHEBI_XXXXXXX`  
  - Only keep objects with the following URI: `obo:MONDO_XXXXXXX`

`gene-disease`:  
- Find all triples that include the predicate: *causes or contributes to* (`obo.RO_0003302`)  
  - Only keep subjects with the following URI: `http://www.ncbi.nlm.nih.gov/gene/X`  
  - Only keep objects with the following URI: `obo:MONDO_XXXXXXX`

In [100]:
def filters_edge_data(graph, relation, subj, obj):
    """Method takes an input RDFLib graph and filters it using an the input relation, subj, and obj variables.
    
    Args:
        graph: An RDFLib Graph object.
        relation: A URIRef object containing a Relation Ontology relation. 
        subj: A URIRef object containing information to filter subjects by.
        obj: A URIRef object containing information to filter objects by.
    
    Returns:
        filtered_edges: 
    """
    
    filtered_triples = []

    for s, p, o in tqdm(graph):
        if p == relation:
            if str(s).startswith(str(subj)) and str(o).startswith(str(obj)):
                filtered_triples += [(s, p, o)]
    
    return filtered_triples

In [101]:
# get gene-drug edges - then keep subjects with 
drug_gene_triples = filters_edge_data(pkt_graph,
                                      obo.RO_0002434,
                                      obo.CHEBI,
                                      URIRef('http://www.ncbi.nlm.nih.gov/gene/'))

print('There are {} drug-gene edges'.format(len(drug_gene_triples)))

100%|██████████| 13438644/13438644 [01:34<00:00, 141873.23it/s]

There are 22418 drug-gene edges


In [107]:
# get drug-disease edges
drug_disease_triples = filters_edge_data(pkt_graph,
                                      obo.RO_0002606,
                                      obo.CHEBI,
                                      obo.MONDO)

print('There are {} drug-disease edges'.format(len(drug_disease_triples)))

100%|██████████| 13438644/13438644 [01:28<00:00, 151497.62it/s]

There are 167850 drug-disease edges


In [108]:
# get gene-disease edges
gene_disease_triples = filters_edge_data(pkt_graph,
                                         obo.RO_0003302,
                                         URIRef('http://www.ncbi.nlm.nih.gov/gene/'),
                                         obo.MONDO)

print('There are {} gene-disease edges'.format(len(drug_disease_triples)))

100%|██████████| 13438644/13438644 [01:30<00:00, 148670.35it/s]

There are 167850 gene-disease edges


In [109]:
# combine triples into single graph
filtered_edges = drug_gene_triples + drug_disease_triples + gene_disease_triples
gene_drug_disease_graph = adds_edges_to_graph(Graph(), filtered_edges)

print('The drug-gene-disease Subgraph contains {} triples'.format(len(gene_drug_disease_graph)))

100%|██████████| 203976/203976 [00:05<00:00, 38867.60it/s]

The drug-gene-disease Subgraph contains 203976 triples


##### Convert Subgraph to NetworkX MultiDiGraph

In [110]:
# convert RDFLib graph to Networkx MultiDiGraph
nx_graph_dgd = nx.MultiDiGraph()

for s, p, o in tqdm(gene_drug_disease_graph):
    nx_graph_dgd.add_edge(s, o, **{'key': p})

100%|██████████| 203976/203976 [00:23<00:00, 8829.97it/s] 


##### Get Graph Descriptives

In [111]:
# get the number of nodes, edges, and self-loops
nodes = nx.number_of_nodes(nx_graph_dgd)
edges = nx.number_of_edges(nx_graph_dgd)
avg_degree = float(edges) / nodes

print('There are {} nodes, {} edges, and has an average degree of {}'.format(nodes, edges, avg_degree))

There are 27159 nodes, 203976 edges, and has an average degree of 7.510438528664531


In [112]:
# get 5 nodes with the highest degress
n_deg = sorted([(str(x[0]), x[1]) for x in  nx_graph_dgd.degree], key=lambda x: x[1], reverse=1)[:6]

for x in n_deg:
    print('{} (degree={})'.format(x[0], x[1]))

http://purl.obolibrary.org/obo/CHEBI_39867 (degree=7920)
http://purl.obolibrary.org/obo/CHEBI_2504 (degree=5717)
http://purl.obolibrary.org/obo/CHEBI_27563 (degree=3270)
http://purl.obolibrary.org/obo/CHEBI_29678 (degree=1248)
http://purl.obolibrary.org/obo/MONDO_0005359 (degree=1112)
http://purl.obolibrary.org/obo/CHEBI_34682 (degree=1063)


In [113]:
# get connected components -- have to convert MultiDiGraph to undirected graph
nx_graph_und = nx_graph_dgd.to_undirected()

# get connected components
components = sorted(list(nx.connected_components(nx_graph_und)), key=len, reverse=True)
cc_content = {x: str(len(components[x])) + ' nodes: ' + ' | '.join(components[x]) if len(components[x]) < 500
              else len(components[x]) for x in range(len(components))}

for k, v in cc_content.items():
    if isinstance(v, int):
        print('COMPONENT: {} Consists of {} nodes'.format(str(k), str(v)))
    else:
        print('\nCOMPONENT: {} Consists of the following nodes:'.format(str(k)))
        for node in v.split(': ')[-1].split(' | '):
            print(node)

COMPONENT: 0 Consists of 26511 nodes

COMPONENT: 1 Consists of the following nodes:
http://purl.obolibrary.org/obo/MONDO_0000736
http://purl.obolibrary.org/obo/MONDO_0012204
http://www.ncbi.nlm.nih.gov/gene/10058
http://purl.obolibrary.org/obo/MONDO_0014169
http://purl.obolibrary.org/obo/MONDO_0013783

COMPONENT: 2 Consists of the following nodes:
http://www.ncbi.nlm.nih.gov/gene/6006
http://purl.obolibrary.org/obo/MONDO_0019107
http://www.ncbi.nlm.nih.gov/gene/6007
http://www.ncbi.nlm.nih.gov/gene/6005
http://purl.obolibrary.org/obo/MONDO_0008493

COMPONENT: 3 Consists of the following nodes:
http://purl.obolibrary.org/obo/MONDO_0014873
http://www.ncbi.nlm.nih.gov/gene/91754
http://purl.obolibrary.org/obo/MONDO_0014870
http://purl.obolibrary.org/obo/MONDO_0013660

COMPONENT: 4 Consists of the following nodes:
http://purl.obolibrary.org/obo/MONDO_0013502
http://www.ncbi.nlm.nih.gov/gene/133396
http://purl.obolibrary.org/obo/MONDO_0024522
http://www.ncbi.nlm.nih.gov/gene/9180

COMPONENT

In [114]:
components[0]

{rdflib.term.URIRef('http://purl.obolibrary.org/obo/MONDO_0005592'),
 rdflib.term.URIRef('http://www.ncbi.nlm.nih.gov/gene/10078'),
 rdflib.term.URIRef('http://www.ncbi.nlm.nih.gov/gene/9987'),
 rdflib.term.URIRef('http://www.ncbi.nlm.nih.gov/gene/100132929'),
 rdflib.term.URIRef('http://www.ncbi.nlm.nih.gov/gene/347744'),
 rdflib.term.URIRef('http://www.ncbi.nlm.nih.gov/gene/27249'),
 rdflib.term.URIRef('http://www.ncbi.nlm.nih.gov/gene/8760'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_28425'),
 rdflib.term.URIRef('http://www.ncbi.nlm.nih.gov/gene/10687'),
 rdflib.term.URIRef('http://www.ncbi.nlm.nih.gov/gene/5692'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/MONDO_0010828'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/MONDO_0003847'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/MONDO_0018952'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/MONDO_0009097'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/MONDO_0017648'),
 rdflib.term.

##### Explore Graph

Example shown below searches for all shortest paths between valporic acid ([CHEBI_39867](https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:39867)) and all other targets in the graph. This search returns several results, one example is shown below:

> **Path between CHEBI_39867 - MONDO_0012489:**  
> http://purl.obolibrary.org/obo/CHEBI_39867  
> http://www.ncbi.nlm.nih.gov/gene/1413  
> http://purl.obolibrary.org/obo/MONDO_0012489  


Where [entrez gene 1413](http://www.ncbi.nlm.nih.gov/gene/1413) is CRYBA4 (crystallin beta A4) and [MONDO_0012489](http://www.ontobee.org/ontology/MONDO?iri=http://purl.obolibrary.org/obo/MONDO_0012489) is early-onset non-syndromic cataract caused by mutation in `CRYBA4`. A quick Googling returns several relevant results to connect these entities.

In [116]:
# perform bidirectional search for path between -- epilepsy and valporic acid
nx.bidirectional_shortest_path(nx_graph_dgd, obo.CHEBI_39867, obo.MONDO_0005027)

[rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_39867'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/MONDO_0005027')]

In [134]:
# look at shortest path between 
shortest_paths = nx.shortest_path(nx_graph_dgd, source=obo.CHEBI_39867)

for k, v in shortest_paths.items():
    if k != obo.CHEBI_39867:
        print('\n{} - {} Path:'.format(str(obo.CHEBI_39867).split('/')[-1], str(k).split('/')[-1]))
        for i in v:
            print(i)


CHEBI_39867 - 400954 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/400954

CHEBI_39867 - 486 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/486

CHEBI_39867 - 51092 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/51092

CHEBI_39867 - 7074 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/7074

CHEBI_39867 - 5992 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/5992

CHEBI_39867 - 133 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/133

CHEBI_39867 - 9659 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/9659

CHEBI_39867 - 6900 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/6900

CHEBI_39867 - 2873 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/2873

CHEBI_39867 - 3880 Path:
http://pu

http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/81894

CHEBI_39867 - MONDO_0014144 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://purl.obolibrary.org/obo/MONDO_0014144

CHEBI_39867 - 10683 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/10683

CHEBI_39867 - 100129396 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/100129396

CHEBI_39867 - 441168 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/441168

CHEBI_39867 - 283507 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/283507

CHEBI_39867 - 83541 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/83541

CHEBI_39867 - 2355 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/2355

CHEBI_39867 - 84065 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/84065

CHEBI_39867 - 10097 P

http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/57582

CHEBI_39867 - 114784 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/114784

CHEBI_39867 - 5837 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/5837

CHEBI_39867 - 7173 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/7173

CHEBI_39867 - 79191 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/79191

CHEBI_39867 - MONDO_0000665 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://purl.obolibrary.org/obo/MONDO_0000665

CHEBI_39867 - 23295 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/23295

CHEBI_39867 - 79646 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/79646

CHEBI_39867 - 23550 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/23550

CHEBI_39867 - 2318 Path:
http://p

http://www.ncbi.nlm.nih.gov/gene/2861

CHEBI_39867 - 767566 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/767566

CHEBI_39867 - 7490 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/7490

CHEBI_39867 - 51182 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/51182

CHEBI_39867 - 1052 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/1052

CHEBI_39867 - 8314 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/8314

CHEBI_39867 - 23239 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/23239

CHEBI_39867 - 10885 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/10885

CHEBI_39867 - 1592 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/1592

CHEBI_39867 - 2287 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov

http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/4125

CHEBI_39867 - 114900 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/114900

CHEBI_39867 - 57099 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/57099

CHEBI_39867 - 57684 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/57684

CHEBI_39867 - 8462 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/8462

CHEBI_39867 - 388730 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/388730

CHEBI_39867 - 6014 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/6014

CHEBI_39867 - 441639 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/441639

CHEBI_39867 - 57678 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/57678

CHEBI_39867 - 89790 Path:
http://purl.obolib

...

http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/3064
http://purl.obolibrary.org/obo/MONDO_0054573

CHEBI_39867 - MONDO_0016621 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/3064
http://purl.obolibrary.org/obo/MONDO_0016621

CHEBI_39867 - MONDO_0007739 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/3064
http://purl.obolibrary.org/obo/MONDO_0007739

CHEBI_39867 - MONDO_0013686 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/859
http://purl.obolibrary.org/obo/MONDO_0013686

CHEBI_39867 - MONDO_0012392 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/36
http://purl.obolibrary.org/obo/MONDO_0012392

CHEBI_39867 - MONDO_0005959 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/10908
http://purl.obolibrary.org/obo/MONDO_0005959

CHEBI_39867 - MONDO_0009514 Path:
http://purl.obolibrary.org/obo/CHEBI

http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/1801
http://purl.obolibrary.org/obo/MONDO_0014824

CHEBI_39867 - MONDO_0013385 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/51082
http://purl.obolibrary.org/obo/MONDO_0013385

CHEBI_39867 - MONDO_0014862 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/120
http://purl.obolibrary.org/obo/MONDO_0014862

CHEBI_39867 - MONDO_0011472 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/5317
http://purl.obolibrary.org/obo/MONDO_0011472

CHEBI_39867 - MONDO_0011888 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/51135
http://purl.obolibrary.org/obo/MONDO_0011888

CHEBI_39867 - MONDO_0014539 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/286204
http://purl.obolibrary.org/obo/MONDO_0014539

CHEBI_39867 - MONDO_0009063 Path:
http://purl.obolibrary.org/obo/

CHEBI_39867 - MONDO_0006012 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/1636
http://purl.obolibrary.org/obo/MONDO_0006012

CHEBI_39867 - MONDO_0018150 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/1636
http://purl.obolibrary.org/obo/MONDO_0018150

CHEBI_39867 - MONDO_0008713 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/55630
http://purl.obolibrary.org/obo/MONDO_0008713

CHEBI_39867 - MONDO_0006523 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/55630
http://purl.obolibrary.org/obo/MONDO_0006523

CHEBI_39867 - MONDO_0014162 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/65080
http://purl.obolibrary.org/obo/MONDO_0014162

CHEBI_39867 - MONDO_0012472 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/10535
http://purl.obolibrary.org/obo/MONDO_0012472

CHEBI_39867 - MONDO_0009629 Pa

CHEBI_39867 - MONDO_0015286 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/10559
http://purl.obolibrary.org/obo/MONDO_0015286

CHEBI_39867 - MONDO_0011342 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/10559
http://purl.obolibrary.org/obo/MONDO_0011342

CHEBI_39867 - MONDO_0010860 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/51168
http://purl.obolibrary.org/obo/MONDO_0010860

CHEBI_39867 - MONDO_0015612 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/1184
http://purl.obolibrary.org/obo/MONDO_0015612

CHEBI_39867 - MONDO_0010644 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/1184
http://purl.obolibrary.org/obo/MONDO_0010644

CHEBI_39867 - MONDO_0010510 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/389856
http://purl.obolibrary.org/obo/MONDO_0010510

CHEBI_39867 - MONDO_0014271 P


CHEBI_39867 - MONDO_0013081 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/3702
http://purl.obolibrary.org/obo/MONDO_0013081

CHEBI_39867 - MONDO_0010339 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/6853
http://purl.obolibrary.org/obo/MONDO_0010339

CHEBI_39867 - MONDO_0011843 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/8557
http://purl.obolibrary.org/obo/MONDO_0011843

CHEBI_39867 - MONDO_0007916 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/1604
http://purl.obolibrary.org/obo/MONDO_0007916

CHEBI_39867 - MONDO_0024535 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/23586
http://purl.obolibrary.org/obo/MONDO_0024535

CHEBI_39867 - MONDO_0014575 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/23586
http://purl.obolibrary.org/obo/MONDO_0014575

CHEBI_39867 - MONDO_0008429 Pat

##### Save Output

In [131]:
# save subgraph
gene_drug_disease_graph.serialize(write_location + 'pkt_DrugGeneDisease_subgraph.nt', format='nt')
nx.write_gpickle(nx_graph_dgd, write_location + 'pkt_DrugGeneDisease_NetworkxMultiDiGraph.gpickle')